This is a temporary notebook

In [1]:
import pandas as pd

In [2]:
# The below code gives a huge error due to encoding issue
# kenta and I have struggled with this for at least an hour...
# df = pd.read_csv("forpractice.csv")

Here we get a huge error due to unicode-decoding error without specifying a decoder in read_csv. So we need to figure out at 0x93 position which decoder we need.

In [3]:
df = pd.read_csv("forpractice.csv", encoding='cp932', skiprows= [0,1,2])

In [4]:
name = pd.read_csv("forpractice.csv", encoding="cp932", nrows=1, header=None)

In [5]:
name.iat[0,0] # this is the name of the individual

'田中敬一郎'

In [6]:
df.head()

,時刻,活動強度,運動種別
0,0:00:00,0.0,計測なし
1,0:01:00,0.0,計測なし
2,0:02:00,0.0,計測なし
3,0:03:00,0.0,計測なし
4,0:04:00,0.0,計測なし


It looks like we did a pretty good job. I don't really understand what the columns are saying so... we should probably change the column names.

In [7]:
df.columns = ["Time","Mets","random"]

Let's also remove the 3rd column because we don't need it

In [8]:
df.drop('random', axis=1, inplace=True)
df.head()

,Time,Mets
0,0:00:00,0.0
1,0:01:00,0.0
2,0:02:00,0.0
3,0:03:00,0.0
4,0:04:00,0.0


Let's convert the time column to datetime type

First create a new column that will convert the charactercolumn `Time` into a timestamp column

In [68]:
df['Time2'] = pd.to_datetime(df['Time'], format="%H:%M:%S")

In [69]:
df.head()

,Time,Mets,Time2
0,0:00:00,0.0,1900-01-01 00:00:00
1,0:01:00,0.0,1900-01-01 00:01:00
2,0:02:00,0.0,1900-01-01 00:02:00
3,0:03:00,0.0,1900-01-01 00:03:00
4,0:04:00,0.0,1900-01-01 00:04:00


Now need to create a column that will subtract all time from the beginning of the day

In [98]:
(df.Time2 - pd.DataFrame({"FTime": [df.Time2.tolist()[0]]*df.shape[0]}).FTime).astype('timedelta64[m]').head()

0    0.0
1    1.0
2    2.0
3    3.0
4    4.0
dtype: float64

In [100]:
df['Time3'] = (df.Time2 - pd.DataFrame({"FTime": [df.Time2.tolist()[0]]*df.shape[0]}).FTime).astype('timedelta64[m]')

In [101]:
df.head(); df.tail()

,Time,Mets,Time2,Time3
1435,23:55:00,0.0,1900-01-01 23:55:00,1435.0
1436,23:56:00,0.0,1900-01-01 23:56:00,1436.0
1437,23:57:00,0.0,1900-01-01 23:57:00,1437.0
1438,23:58:00,0.0,1900-01-01 23:58:00,1438.0
1439,23:59:00,0.0,1900-01-01 23:59:00,1439.0


Finally we have a column that has minutes from the beginning to the end

So we need to find the count of individuals that have the `Mets` column >= 0. We need to determine if the count is > 150 (after aggregating all the 7 days)

In [102]:
df.loc[df.Mets >= 3].Mets.count()

8

In [103]:
df.loc[(df['Mets']>= 1.6) & (df['Mets'] < 3)].Mets.count() #LIPA

215

In [104]:
df.loc[(df['Mets'] <= 1.5)].Mets.count() #Sedentary 1217

1217

Now we need to add Short-bout Long-bout MVPA where SB is 1 ~ 9 mins MVPA and LB is >= 10 mins MVPA. We need to sum the count of the Mets.

In [105]:
df.loc[df.Mets >= 3].shape[0]

8

The person above has no consecutive MVPA thus would be considered to have 8 mins of SB MVPA.

In [106]:
df.loc[df.Mets >= 3]

,Time,Mets,Time2,Time3
527,8:47:00,3.1,1900-01-01 08:47:00,527.0
568,9:28:00,3.1,1900-01-01 09:28:00,568.0
691,11:31:00,3.1,1900-01-01 11:31:00,691.0
835,13:55:00,3.0,1900-01-01 13:55:00,835.0
838,13:58:00,3.0,1900-01-01 13:58:00,838.0
845,14:05:00,3.1,1900-01-01 14:05:00,845.0
846,14:06:00,3.2,1900-01-01 14:06:00,846.0
884,14:44:00,3.0,1900-01-01 14:44:00,884.0


In [108]:
(df.loc[df.Mets >= 3].iat[6,2] - df.loc[df.Mets >= 3].iat[5,2]).seconds > 60

False

In the above case, the person has had one consecutive MVPA minutes (i.e. 2 minutes) but otherwise they were all less than 9 minutes therefore these 8 minutes were all SB MVPA. The index number 845 and 846 indicates that time 845 and 846 were consecutive MVPA values.

We are going to use the `itertools` package `groupby` class to sort rows into consecutive times. First let's just convert the times into pure numbers so it's easier for us to count using the `itertools`

In [129]:
from itertools import groupby, accumulate
from operator import itemgetter, mul
import operator

In [189]:
len(data)

14

In [190]:
for k, g in groupby(enumerate(df.loc[df.Mets >= 3].Time3.tolist()), lambda i: i[0] - i[1]):
    print(len(list(map(itemgetter(1), g))))

1
1
1
1
1
2
1


In [218]:
for key, group in groupby(enumerate(data), lambda i: i[0] - i[1]):
    group = list(map(itemgetter(1), group))
    ranges.append((group[-1]-group[0]))

The above code prints the list of consecutive times of MVPA. Now we will just create a function that will take a list and output a list of two values. The two values will be the sum of consecutive Long-bout and Short-bout MVPA times. For someone who had a MVPA value of >= 10 for two minute should have a value of 2 for the LB_MVPA value while if that same person had 20 minutes of MVPA values < 10, then that person should have a SB_MVPA value of 20 therefore the list would look like `[2, 20]` or some form that has two values.

In [232]:
def mvpa_sblb(list_data):
    """
    The input should be a list!
    This function returns a list of 2 values. 1 value is the number of short-bout MVPA 
    The other is the number of long-bout MVPA. 
    """
    
    #  first create empty list that will be our output 
    ranges = []

    # we need to iterate through the list. This for loop below will create a 
    # list of values that indicate the consecutive MVPA minutes. 
    # Therefore a number 2 in the list will indicate that
    # The participant had 2 minutes of consecutive time during the day(or week).
    # many thanks to https://stackoverflow.com/questions/2154249/identify-groups-of-continuous-numbers-in-a-list
    
    for key, group in groupby(enumerate(list_data), lambda i: i[0] - i[1]):
        group = list(map(itemgetter(1), group))
        ranges.append((group[-1]-group[0]))
        
    # Now we need to check the list and sum the values that are > 9 and that forms the value for
    # lb_mvpa. If values are <= 9 then they sum to be the value for sb_mvpa
    sb_list = []
    lb_list = []
    for i in ranges:
        if ( i >= 10 ):
            lb_list.append((i))
        else:
            sb_list.append((i))
    
    # sum the values in each list and return list
    finlist = {"SB": sum(sb_list), 
               "LB":sum(lb_list)}
    
    return(finlist)

In [233]:
mvpa_sblb(data)

{'LB': 0, 'SB': 8}